In [1]:
import tensorflow.compat.v1 as tf
from tensorflow.compat.v1.keras.models import load_model
import tensorflow.compat.v1.keras.backend as K
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.colors import ListedColormap

In [3]:
import cv2
import glob
import time
import gc

## Function

In [4]:
def get_heatmap_matrix(dataset, model, nth_data, layer, verbose = False):

    #Get the data
    data = dataset[nth_data:nth_data+1, :, :]
    if verbose: print(data.shape)

    # Get the prediction from the model
    preds = model.predict(data)

    # Get the position that maximally activated
    position = np.argmax(preds[0])
    if verbose: print(position)

    # Get the output of the layer and the data
    data_output = model.output[:, 0]

    # Get layer output
    get_layer = model.get_layer(layer)
    layer_output = get_layer.output
    num_filter = layer_output.shape[2]

    # Calculate the gradients
    grads = K.gradients(data_output, layer_output)[0]
    if verbose: print(grads.shape)

    pooled_grads = K.mean(grads, axis = (0, 1))

    iterate = K.function([model.input], [pooled_grads, layer_output[0]])

    if verbose: print("Starting K iteration...")
    t = time.time()
    pooled_grads_value, layer_output_value = iterate([data])
    if verbose: print("Finished K iteration " + "(" + str(time.time() - t) + "seconds)")
    if verbose: print(layer_output_value.shape)

    if verbose: print("Starting loop...")
    t = time.time()
    # Looping
    for i in range(num_filter):
        layer_output_value[:, i] *= pooled_grads_value[i]

    if verbose: print("Loop finished " + " (" + str(time.time() - t) + "seconds)" )
    # Get the heatmap matrix
    heatmap = np.mean(layer_output_value, axis = -1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    # Expand the dimensionality of heatmap so that it can be plot
    heatmap = np.expand_dims(heatmap, axis=0)

    return heatmap

In [5]:
def load_npy_data(pathogen_class, filename):
    # Define pattern of the npy data
    data_loading_pattern = "data-sets/" + pathogen_class + "/" + filename + ".npy"
    data_loading_path = glob.glob(data_loading_pattern)

    # Load the data
    print("Loading data:", str(filename), "from " + data_loading_path[0])
    data = np.load(data_loading_path[0])
    
    return(data)

In [6]:
def get_sum_heatmap(pathogen_class, data_name, model, layer):
    
    data = load_npy_data(pathogen_class, data_name)
    result = []

    max_sample = data.shape[0]
    # max_sample = 1 
    
    print("")
    # For loop to get the heatmap for each matrix
    for nth_sample in range(0, max_sample):
        print("Getting the heatmap for the data: sample " + str(nth_sample + 1) + "/" + str(max_sample))
        heatmap = get_heatmap_matrix(data, model, nth_sample, layer, verbose = False)

        # Making sure that the shape is correct which is (1, 4034)
        print("Heatmap size:" + str(heatmap.shape))

        # Put all of the reults together
        result.append(heatmap)
        
        # Free memory
        gc.collect()

    # Change the list to numpy array
    all_matrices = np.array(result)

    # Get sum of all matrices
    sum_all_matrices = np.sum(all_matrices, axis = 0)

    # Save the sum of all matrices
    saving_path = "results/sum_all_matrices_" + pathogen_class + "_" + data_name + "_" + layer + ".npy"

    print("")
    print("Saving results in", saving_path)
    np.save(saving_path, sum_all_matrices)

## Load all of the model

In [7]:
# model_cnn_gru_bacteria = load_model("models/bacteria/cnn-gru/sequential_1.30-0.27.hdf5")
# model_cnn_gru_fungi = load_model("models/fungi/cnn-gru/sequential_1.30-0.27.hdf5")
model = load_model("models/fungi/cnn-lstm/model_1.30-0.28.hdf5")
# model_cnn_gru_oomycete = load_model("models/oomycete/cnn-gru/sequential_1.30-0.37.hdf5")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [8]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4034, 20)]   0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 4034, 4)      80          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 4032, 4)      240         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 4030, 4)      400         input_1[0][0]                    
____________________________________________________________________________________________

In [9]:
# bacteria 
# get_sum_heatmap("bacteria", "x_train", model_cnn_gru_bacteria, "conv1d_1")
# get_sum_heatmap("bacteria", "x_val", model_cnn_gru_bacteria, "conv1d_1")
# get_sum_heatmap("bacteria", "x_test", model_cnn_gru_bacteria, "conv1d_1")
get_sum_heatmap("fungi", "x_test", model, "conv1d_4")

Loading data: x_test from data-sets/fungi/x_test.npy

Getting the heatmap for the data: sample 1/44
Heatmap size:(1, 12094)
Getting the heatmap for the data: sample 2/44
Heatmap size:(1, 12094)
Getting the heatmap for the data: sample 3/44
Heatmap size:(1, 12094)
Getting the heatmap for the data: sample 4/44
Heatmap size:(1, 12094)
Getting the heatmap for the data: sample 5/44
Heatmap size:(1, 12094)
Getting the heatmap for the data: sample 6/44
Heatmap size:(1, 12094)
Getting the heatmap for the data: sample 7/44
Heatmap size:(1, 12094)
Getting the heatmap for the data: sample 8/44
Heatmap size:(1, 12094)
Getting the heatmap for the data: sample 9/44
Heatmap size:(1, 12094)
Getting the heatmap for the data: sample 10/44
Heatmap size:(1, 12094)
Getting the heatmap for the data: sample 11/44
Heatmap size:(1, 12094)
Getting the heatmap for the data: sample 12/44
Heatmap size:(1, 12094)
Getting the heatmap for the data: sample 13/44
Heatmap size:(1, 12094)
Getting the heatmap for the data